# QATNE Hardware Experiments

This notebook demonstrates running QATNE on real IBM Quantum hardware.

## Prerequisites
- IBM Quantum account with API token
- Access to quantum devices (free tier sufficient for small molecules)

In [ ]:
# Install required packages
!pip install -q qiskit qiskit-ibm-runtime qiskit-aer
!pip install -q openfermion pyscf
!pip install -q matplotlib seaborn numpy scipy

## 1. Setup IBM Quantum Connection

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Save your IBM Quantum token
# Get token from: https://quantum.ibm.com/
IBM_TOKEN = "YOUR_IBM_QUANTUM_TOKEN_HERE"

# Initialize service
try:
    service = QiskitRuntimeService(
        channel='ibm_quantum',
        token=IBM_TOKEN
    )
    print("✓ Connected to IBM Quantum")
except Exception as e:
    print(f"Error connecting: {e}")
    print("
Using simulator instead...")
    service = None

## 2. Select Quantum Backend

In [ ]:
if service is not None:
    # Get available backends
    backends = service.backends(
        filters=lambda x: x.configuration().n_qubits >= 5
                         and not x.configuration().simulator
    )
    
    print("Available quantum devices:
")
    for backend in backends[:5]:  # Show first 5
        config = backend.configuration()
        status = backend.status()
        print(f"  {config.backend_name}:")
        print(f"    Qubits: {config.n_qubits}")
        print(f"    Pending jobs: {status.pending_jobs}")
        print(f"    Operational: {status.operational}
")
    
    # Select least busy backend
    backend = service.least_busy(
        filters=lambda x: x.configuration().n_qubits >= 5
    )
    print(f"
✓ Selected backend: {backend.name}")
else:
    from qiskit_aer import AerSimulator
    backend = AerSimulator()
    print("✓ Using AerSimulator (no IBM connection)")

## 3. Prepare H2 Molecule

In [ ]:
from openfermion import MolecularData, jordan_wigner
from openfermionpyscf import run_pyscf

def create_h2_hamiltonian(bond_length=0.735):
    """Create H2 Hamiltonian for quantum hardware"""
    geometry = [('H', (0, 0, 0)), ('H', (0, 0, bond_length))]
    
    molecule = MolecularData(
        geometry=geometry,
        basis='sto-3g',
        multiplicity=1,
        charge=0
    )
    
    molecule = run_pyscf(molecule, run_scf=True, run_fci=True)
    hamiltonian_ferm = molecule.get_molecular_hamiltonian()
    hamiltonian_jw = jordan_wigner(hamiltonian_ferm)
    
    return hamiltonian_jw.to_matrix(), molecule.fci_energy

H_matrix, exact_energy = create_h2_hamiltonian()
num_qubits = int(np.log2(H_matrix.shape[0]))

print(f"H2 Molecule Prepared")
print(f"  Qubits required: {num_qubits}")
print(f"  Exact FCI energy: {exact_energy:.8f} Ha")

## 4. Initialize QATNE with Hardware Backend

In [ ]:
import sys
sys.path.append('..')

# Import QATNE components
from qatne.algorithms.qatne_solver import QATNESolver
from qatne.algorithms.error_mitigation import ZNEMitigator

# Initialize solver with hardware backend
solver = QATNESolver(
    hamiltonian=H_matrix,
    num_qubits=num_qubits,
    max_bond_dim=8,  # Smaller for hardware
    convergence_threshold=1e-4,
    shots=4096  # Adjust based on backend availability
)

# Use custom backend
solver.backend = backend

print(f"✓ QATNE initialized with {backend.name}")
print(f"  Maximum bond dimension: 8")
print(f"  Shots per circuit: 4096")

## 5. Error Mitigation Setup

In [ ]:
# Initialize Zero-Noise Extrapolation
mitigator = ZNEMitigator(
    noise_factors=[1.0, 1.5, 2.0],
    extrapolation_method='polynomial'
)

print("✓ Error mitigation configured")
print(f"  Method: Zero-Noise Extrapolation (ZNE)")
print(f"  Noise factors: {mitigator.noise_factors}")

## 6. Run Hardware Experiment

In [ ]:
import time

print("Starting hardware experiment...")
print("This may take several minutes depending on queue time.
")

start_time = time.time()

# Run optimization with fewer iterations for hardware
try:
    ground_energy, optimal_params = solver.solve(
        max_iterations=50,  # Reduced for hardware
        optimizer='adam'
    )
    
    # Apply error mitigation
    mitigated_energy = mitigator.mitigate(ground_energy, solver)
    
    end_time = time.time()
    
    print("
" + "="*60)
    print("HARDWARE EXPERIMENT RESULTS")
    print("="*60)
    print(f"Raw energy:        {ground_energy:.8f} Ha")
    print(f"Mitigated energy:  {mitigated_energy:.8f} Ha")
    print(f"Exact energy:      {exact_energy:.8f} Ha")
    print(f"Raw error:         {abs(ground_energy - exact_energy):.2e} Ha")
    print(f"Mitigated error:   {abs(mitigated_energy - exact_energy):.2e} Ha")
    print(f"Total time:        {end_time - start_time:.1f} seconds")
    print(f"Backend:           {backend.name}")
    
except Exception as e:
    print(f"
Error during execution: {e}")
    print("This may be due to backend availability or API limits.")

## 7. Analyze Hardware Noise

In [ ]:
if service is not None and hasattr(backend, 'properties'):
    props = backend.properties()
    
    # Get qubit properties
    t1_times = []
    t2_times = []
    readout_errors = []
    
    for qubit in range(min(num_qubits, backend.configuration().n_qubits)):
        t1 = props.t1(qubit)
        t2 = props.t2(qubit)
        readout = props.readout_error(qubit)
        
        if t1 is not None:
            t1_times.append(t1)
        if t2 is not None:
            t2_times.append(t2)
        readout_errors.append(readout)
    
    # Plot noise characteristics
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    axes[0].bar(range(len(t1_times)), t1_times, color='skyblue')
    axes[0].set_xlabel('Qubit')
    axes[0].set_ylabel('T1 (μs)')
    axes[0].set_title('Relaxation Time (T1)')
    axes[0].grid(True, alpha=0.3)
    
    axes[1].bar(range(len(t2_times)), t2_times, color='lightcoral')
    axes[1].set_xlabel('Qubit')
    axes[1].set_ylabel('T2 (μs)')
    axes[1].set_title('Dephasing Time (T2)')
    axes[1].grid(True, alpha=0.3)
    
    axes[2].bar(range(len(readout_errors)), readout_errors, color='lightgreen')
    axes[2].set_xlabel('Qubit')
    axes[2].set_ylabel('Error Rate')
    axes[2].set_title('Readout Error')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('hardware_noise_profile.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"
Hardware Noise Characteristics:")
    print(f"  Average T1: {np.mean(t1_times):.2f} μs")
    print(f"  Average T2: {np.mean(t2_times):.2f} μs")
    print(f"  Average readout error: {np.mean(readout_errors):.4f}")

## 8. Compare Simulator vs Hardware

In [ ]:
from qiskit_aer import AerSimulator

# Run same experiment on simulator for comparison
print("Running comparison on ideal simulator...")

solver_sim = QATNESolver(
    hamiltonian=H_matrix,
    num_qubits=num_qubits,
    max_bond_dim=8,
    convergence_threshold=1e-4,
    shots=4096
)
solver_sim.backend = AerSimulator()

sim_energy, _ = solver_sim.solve(max_iterations=50, optimizer='adam')

# Comparison table
print("
" + "="*60)
print("SIMULATOR VS HARDWARE COMPARISON")
print("="*60)
print(f"{'Method':<20} {'Energy (Ha)':<15} {'Error (Ha)':<15}")
print("-" * 60)
print(f"{'Exact (FCI)':<20} {exact_energy:<15.8f} {0.0:<15.2e}")
print(f"{'Simulator':<20} {sim_energy:<15.8f} {abs(sim_energy - exact_energy):<15.2e}")
if 'ground_energy' in locals():
    print(f"{'Hardware (raw)':<20} {ground_energy:<15.8f} {abs(ground_energy - exact_energy):<15.2e}")
    print(f"{'Hardware (ZNE)':<20} {mitigated_energy:<15.8f} {abs(mitigated_energy - exact_energy):<15.2e}")
print("="*60)

## 9. Visualization: Energy Landscape

In [ ]:
# Plot convergence comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Simulator convergence
ax = axes[0]
ax.plot(solver_sim.energy_history, 'b-', linewidth=2, label='Simulator')
ax.axhline(exact_energy, color='r', linestyle='--', linewidth=2, label='Exact')
ax.set_xlabel('Iteration')
ax.set_ylabel('Energy (Ha)')
ax.set_title('Simulator Convergence')
ax.legend()
ax.grid(True, alpha=0.3)

# Hardware convergence (if available)
ax = axes[1]
if 'solver' in locals() and hasattr(solver, 'energy_history'):
    ax.plot(solver.energy_history, 'g-', linewidth=2, label='Hardware')
ax.axhline(exact_energy, color='r', linestyle='--', linewidth=2, label='Exact')
ax.set_xlabel('Iteration')
ax.set_ylabel('Energy (Ha)')
ax.set_title('Hardware Convergence')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('hardware_vs_simulator.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Save Results

In [ ]:
import json
from datetime import datetime

results = {
    'timestamp': datetime.now().isoformat(),
    'molecule': 'H2',
    'bond_length': 0.735,
    'backend': backend.name,
    'num_qubits': num_qubits,
    'shots': 4096,
    'exact_energy': float(exact_energy),
    'simulator_energy': float(sim_energy),
}

if 'ground_energy' in locals():
    results['hardware_energy_raw'] = float(ground_energy)
    results['hardware_energy_mitigated'] = float(mitigated_energy)
    results['hardware_error_raw'] = float(abs(ground_energy - exact_energy))
    results['hardware_error_mitigated'] = float(abs(mitigated_energy - exact_energy))

# Save to file
with open('hardware_experiment_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("
✓ Results saved to 'hardware_experiment_results.json'")
print("
" + "="*60)
print("HARDWARE EXPERIMENT COMPLETE!")
print("="*60)